In [0]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -U rasa_core==0.9.6 rasa_nlu[spacy];

In [0]:
!python -m spacy download en_core_web_md
!python -m spacy link en_core_web_md en --force;

## Define the Stories

In [0]:
stories_md = """
## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye
"""
%store stories_md > stories.md

print("Done!")

## Define the Domain

In [0]:
domain_yml = """
intents:
  - greet
  - goodbye
  - mood_affirm
  - mood_deny
  - mood_great
  - mood_unhappy

actions:
- utter_greet
- utter_cheer_up
- utter_did_that_help
- utter_happy
- utter_goodbye

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
"""
%store domain_yml > domain.yml

print("Done!")

## Train RASA Core

In [0]:
!python -m rasa_core.train -d domain.yml -s stories.md -o models/dialogue
print("Finished training!")

## Create the dataset for NLU

In [0]:

nlu_md = """
## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad
- I feel blue
"""
%store nlu_md > nlu.md

print("Done!")

## Define the RASA Pipeline

In [0]:
nlu_config = """
language: en
pipeline: spacy_sklearn
"""
%store nlu_config > nlu_config.yml

print("Done!")

## Train an NLU model 

In [0]:
!python -m rasa_nlu.train -c nlu_config.yml --data nlu.md -o models --fixed_model_name nlu --project current --verbose

## Talking To Your Bot

In [0]:
import IPython
from IPython.display import clear_output, HTML, display
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
import time

In [0]:
interpreter = RasaNLUInterpreter('models/current/nlu')
agent = Agent.load('models/dialogue', interpreter=interpreter)
messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]

def chatlogs_html(messages):
    messages_html = "".join(["<p>{}</p>".format(m) for m in messages])
    chatbot_html = """<div class="chat-window" {}</div>""".format(messages_html)
    return chatbot_html


while True:
    clear_output()
    display(HTML(chatlogs_html(messages)))
    time.sleep(0.3)    
    a = input()
    messages.append(a)
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for r in responses:
        messages.append(r.get("text"))

## Custom Actions

In [0]:
!pip install datetime
!pip install timedelta

## Define the custom function

In [0]:
custom_action = """

from rasa_core.actions import Action
import numpy as np
from IPython.core.display import Image, display
from datetime import datetime, timedelta


import requests

descr = {
    'clear sky': 0,
    'few clouds': 1,
    'light rain': 2,
    'moderate rain': 3,
    'broken clouds': 4,
    'scattered clouds': 5,
    'overcast clouds': 6
}

week_dict = {
    'sunday': 5,
    'monday': 6,
    'tuesday': 0,
    'wednesday': 1,
    'thursday': 2,
    'friday': 3,
    'saturday': 4
}


class ActionWeatherRequest(Action):
    def name(self):
        return "action_weather_request"

    def run(self, dispatcher, tracker, domain):
        
      weekday = next(tracker.get_latest_entity_values("weekday"), None)
      print("Received:",weekday)

      r = requests.get('http://api.openweathermap.org/data/2.5/forecast?id=3467865&APPID=10b05519ce75e9322480d20bf03e817c&units=metric')
      if r.status_code == 200:
         data = r.json()
         avg_temp = {}
         descr_temp = {}

         for d in data['list']:

            ts = int(d['dt'])

            # if you encounter a "year is out of range" error the timestamp
            # may be in milliseconds, try `ts /= 1000` in that case
            date_str = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

            if date_str not in avg_temp:
               avg_temp[date_str] = [d['main']['temp']]
               descr_temp[date_str] = [d['weather'][0]['description']]
            else:
               avg_temp[date_str].append(d['main']['temp'])
               descr_temp[date_str].append(d['weather'][0]['description'])

         avg_temp = {key: np.mean(values) for key, values in avg_temp.items()}

         now = datetime.now()
         weekday_id = int(now.utcfromtimestamp(ts).strftime('%w'))  # 0 --> Sunday
         week_dict['today'] = weekday_id
         week_dict['tomorrow'] = weekday_id + 1

         increase_n_days = week_dict[weekday] - weekday_id

         if increase_n_days < 0:
            message = "Sorry I can only look at the future."
         elif increase_n_days > 5:
            message = "I cannot look that far in the future."
         else:
            now = now + timedelta(days=increase_n_days)
            date = now.strftime('%Y-%m-%d')

            if descr_temp[date][0] != descr_temp[date][-1]:
               message = "The average temperature for {0} is {1:.1f}. The day starts off with {2} but ends with a {3}".format(
            weekday, avg_temp[date], descr_temp[date][0], descr_temp[date][-1])
            else:
               message = "The average temperature for {0} is {1:.1f}. Most of the day will have {2}.".format(weekday,
                                                                                                avg_temp[date],
                                                                                                descr_temp[
                                                                                                   date][0])

         dispatcher.utter_message(message)
      else:
         dispatcher.utter_message('I am having problems with internet connection...')

"""


%store custom_action > actions.py

In [0]:
! pip install rasa_core_sdk

## Adding new Story

In [0]:
stories_md = """
## weather request
* weather_request{"weekday": "friday"}
  - action_weather_request
"""
%store stories_md >> stories.md

print("Done!")

## Redifine our Domain

In [0]:
domain_yml = """
intents:
  - greet
  - goodbye
  - mood_affirm
  - mood_deny
  - mood_great
  - mood_unhappy
  - weather_request

actions:
- utter_greet
- utter_cheer_up
- utter_did_that_help
- utter_happy
- utter_goodbye
- actions.ActionWeatherRequest

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
  
entities:
  - weekday
"""
%store domain_yml > domain.yml

print("Done!")

## Adding new training data

In [0]:
nlu_md = """
## intent:weather_request
- how is the weather [tomorrow](weekday) <!-- entity "weekday" has value "tomorrow" -->
- show me the weather for [today](weekday)
- how's the weather next [monday](weekday)
- what is the temperature next [friday](weekday)
- what's the weather [today](weekday)
- tell me the weather for next [tuesday](weekday)
- how's the temperature for [friday](weekday)
"""
%store nlu_md >> nlu.md

print("Done!")

In [0]:
!python -m rasa_core.train -d domain.yml -s stories.md -o models/dialogue
print("Finished training core!")

In [0]:
!python -m rasa_nlu.train -c nlu_config.yml --data nlu.md -o models --fixed_model_name nlu --project current --verbose
print("Finished training NLU!")

In [0]:
! python3 -m rasa_core_sdk.endpoint --actions actions &
# need to cancel this cell, the process runs on background

## Talk to your bot

In [0]:
interpreter = RasaNLUInterpreter('models/current/nlu')
agent = Agent.load('models/dialogue', interpreter=interpreter)
messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]

def chatlogs_html(messages):
    messages_html = "".join(["<p>{}</p>".format(m) for m in messages])
    chatbot_html = """<div class="chat-window" {}</div>""".format(messages_html)
    return chatbot_html


while True:
    clear_output()
    display(HTML(chatlogs_html(messages)))
    time.sleep(0.3)    
    a = input()
    messages.append(a)
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    print(responses)
    for r in responses:
        messages.append(r.get("text"))